In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchsummary import summary
from torch.utils.data import DataLoader
from torchvision import datasets
import numpy as np
import random
from model_structure import get_preprocessing_transforms, train_model, evaluate_model, get_resnet_model, get_squ_model

In [2]:
seed = 441
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
np.random.seed(seed)
random.seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [3]:
MODEL_NUMBER = 2
BATCH_SIZE = 32
LEARNING_RATE = 0.001
NUM_EPOCHS = 50
INPUT_SIZE = 224
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [4]:
# Get transforms
train_transform, val_transform = get_preprocessing_transforms(INPUT_SIZE)

In [5]:
# master file path 
master_file_path = '.'
# Set data set directory
train_dir = f'{master_file_path}/data4model_{MODEL_NUMBER}/train/'
val_dir = f'{master_file_path}/data4model_{MODEL_NUMBER}/test/'

In [6]:
# Create datasets
train_dataset = datasets.ImageFolder(
        root=train_dir,
        transform=train_transform
    )
    
val_dataset = datasets.ImageFolder(
        root=val_dir,
        transform=val_transform
    )

In [7]:
# Create data loaders
train_loader = DataLoader(
    train_dataset, 
    batch_size=BATCH_SIZE, 
    shuffle=True, 
    num_workers=4,
    pin_memory=True
)

val_loader = DataLoader(
    val_dataset, 
    batch_size=BATCH_SIZE, 
    shuffle=False, 
    num_workers=4,
    pin_memory=True
)

In [8]:
model = get_resnet_model(2).to(DEVICE)
criterion =  nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)
model.eval()

/home/hsk/venv/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/hsk/venv/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [9]:
summary(model, (3, INPUT_SIZE, INPUT_SIZE))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 112, 112]           9,408
       BatchNorm2d-2         [-1, 64, 112, 112]             128
              ReLU-3         [-1, 64, 112, 112]               0
         MaxPool2d-4           [-1, 64, 56, 56]               0
            Conv2d-5           [-1, 64, 56, 56]          36,864
       BatchNorm2d-6           [-1, 64, 56, 56]             128
              ReLU-7           [-1, 64, 56, 56]               0
            Conv2d-8           [-1, 64, 56, 56]          36,864
       BatchNorm2d-9           [-1, 64, 56, 56]             128
             ReLU-10           [-1, 64, 56, 56]               0
       BasicBlock-11           [-1, 64, 56, 56]               0
           Conv2d-12           [-1, 64, 56, 56]          36,864
      BatchNorm2d-13           [-1, 64, 56, 56]             128
             ReLU-14           [-1, 64,

In [10]:
tracker = train_model(model, train_loader, val_loader, criterion, optimizer, NUM_EPOCHS, DEVICE, MODEL_NUMBER)

Epoch [1/50]
Training Loss: 0.2309, Validation Loss: 1.9616
Training Acc: 0.9097, Validation Acc: 0.5200
Training F1: 0.9104, Validation F1: 0.0769
Epoch [2/50]
Training Loss: 0.1315, Validation Loss: 0.8656
Training Acc: 0.9473, Validation Acc: 0.7500
Training F1: 0.9476, Validation F1: 0.6753
Epoch [3/50]
Training Loss: 0.1085, Validation Loss: 0.5396
Training Acc: 0.9640, Validation Acc: 0.8150
Training F1: 0.9642, Validation F1: 0.7836
Epoch [4/50]
Training Loss: 0.0926, Validation Loss: 0.4002
Training Acc: 0.9633, Validation Acc: 0.8350
Training F1: 0.9635, Validation F1: 0.8114
Epoch [5/50]
Training Loss: 0.0956, Validation Loss: 0.6180
Training Acc: 0.9633, Validation Acc: 0.8050
Training F1: 0.9635, Validation F1: 0.7821
Epoch [6/50]
Training Loss: 0.0733, Validation Loss: 1.9492
Training Acc: 0.9743, Validation Acc: 0.5950
Training F1: 0.9744, Validation F1: 0.3306
Epoch [7/50]
Training Loss: 0.0930, Validation Loss: 0.6528
Training Acc: 0.9680, Validation Acc: 0.8050
Trainin